# Exercise 1.6.1 — Fully Convolutional Networks
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree offered at Udacity.

## Objectives

* TODO.

## 1. Introduction

In [ ]:
### Importing the required modules

In [ ]:
import numpy as np
import os
import tensorflow as tf
from typing import List, Union, Tuple

In [ ]:
tf.__version__

'2.9.2'

In [ ]:
tf.test.gpu_device_name()

''

In [ ]:
### Setting the environment variables

In [ ]:
ENV_COLAB = True                # True if running in Google Colab instance

In [ ]:
# Root directory
DIR_BASE = '' if not ENV_COLAB else '/content/'

In [ ]:
# Subdirectory to save output files
DIR_OUT = os.path.join(DIR_BASE, 'out/')
# Subdirectory pointing to input data
DIR_SRC = os.path.join(DIR_BASE, 'data/')

In [ ]:
### Creating subdirectories (if not exists)
os.makedirs(DIR_OUT, exist_ok=True)

### 1.1. Fully Convolutional Networks

TODO.

#### From Fully-Connected to Fully-Convolutional Layers

In this task, we rewrite a Dense fully-connected layer (`tf.keras.layers.Dense`) as a 2D Convolutional layer (`tf.keras.layers.Conv2D`). To do so, we follow the simple guidelines for converting a fully-connected to a fully-convolutional layer:
   * The number of _outputs_ of the fully-connected layer becomes the _kernel size_ of the fully-convolutional layer;
   * The number of _inputs_ of the fully-connected layer becomes the number of _weights_ of the fully-convolutional layer.
    
In addition to the above rules-of-thumb, we also enforce the following specifications for our 1x1 2D Convolutional layer:
   * **Filter size**: $1\times 1$;
   * **Stride**: $1$;
   * **Padding**: Zero-padding.
   
Note that the filter size is used interchangeably with **kernel size** here. By setting our filter to a size of $1\times 1$, we are analogising fully-connected layers with fully-convolutional layers; however, with this convolutional layer we introduce the ability of the network to preserve spatial information in the input tensor. This is in contrast with the Dense fully-connected layer which does not have the ability to preserve spatial information, since it flattens the input image into a one-dimensional vector. 

The ability to preserve 2D spatial information can be used to identify features in the input image and make predictions more accurately when compared to the original Fully Connected Network. Additionally, the use of a $1\times 1$ convolutional layer can reduce the number of parameters in the network and therefore make it more computationally-efficient. By choosing a filter size of $1\times 1$, we get the ability to reduce network parameters while maintaining a significant amount of spatial information throughout the network, which might not be the case when selecting larger filter sizes (e.g., $3\times 3$ or $5\times 5$). 

### 1.2. Transposed Convolutions

TODO.

## 2. Programming Task

NOTE: the code provided here has been migrated to the TensorFlow 2.x API. Some functionality may differ from the original implementation.

### 2.1. Fully Convolutional Layer

In [ ]:
### From Udacity's `quiz.py`

In [ ]:
# custom init with the seed set to 0 by default
def custom_init(
        shape: Union[tf.Tensor, List[int], Tuple[int]], 
        dtype: tf.dtypes.DType=tf.float32,
        seed: int=0,
        partition_info=None
) -> tf.Tensor:
    """Initialises the weights of a layer.
    
    Samples the values at random from a parameterised normal distribution.
    
    :param shape: Shape of the weight vector (i.e., number of weights).
    :param dtype: Data type of the weight vector values to return.
    :param seed: Value of the random seed to create.
    :param partition_info: Optional info about paritioning of a tensor,
        not used in TF2.x API.
    :returns: weight vector of randomly initialised values.
    """
    return tf.random.normal(
        shape=shape, 
        dtype=dtype, 
        seed=seed
    )


def conv_1x1(
        filters: int,
        kernel_size: Union[List[int], Tuple[int], tf.Tensor]=(1, 1),
        stride: int=1
) -> tf.Tensor:
    """Initialises a 1x1 2D Convolutional layer.
    
    To convert a fully-connected to a fully-convolutional layer, we initialise
    the 2D Convolutional layer parameters according to the following:
       1. The number of outputs becomes the kernel size,
       2. The number of inputs becomes the number of weights.
    
    NOTE: The `tf.layers.conv2d` API has been deprecated since TF1.15,
    therefore we use the `tf.keras.layers.Conv2D` layer in TF2.x to initialise
    the layer with modified arguments.
    
    :param filters: the dimensions of the output.
    :param kernel_size: the dimensions of the kernel, i.e., size of the window
    used in the convolution / sliding window operations.
    :param stride: the amount of pixels to "shift" the filter over the input on
    each sliding window operation.
    :returns: the configured `Conv2D` layer.
    """
    return tf.keras.layers.Conv2D(
        filters=filters,
        kernel_size=kernel_size,
        strides=stride,
        padding='VALID',
        kernel_initializer=custom_init
    )

#### Testing the FCN layer

In [ ]:
### Setting the parameters
# Number of output channels (i.e., number of kernels)
NUM_OUTPUTS = 2
KERNEL_SIZE = (1, 1)
# Number of pixels to "move over" for each sliding window operation
STRIDE = (1, 1)
# Batch size (i.e., number of samples per iteration)
BATCH_SIZE = 1
# Number of input channels
CHANNELS_IN = 1

In [ ]:
### Creating an input tensor

In [ ]:
x = tf.convert_to_tensor(
    np.random.randn(BATCH_SIZE, NUM_OUTPUTS, NUM_OUTPUTS, CHANNELS_IN), 
    dtype=tf.float32
)
x

The above "dataset" is defined as a 4-D tensor with a number of samples equal to one (i.e., `BATCH_SIZE = 1`). Since we are going to be comparing the effect of the `Conv2D` layer to the `Dense` layer on the output image size, we want to set our input "image" to be of size (`NUM_OUTPUT`, `NUM_OUTPUT`, `CHANNELS_IN`). That is, we are expecting the input and output size of the tensor to be the same (unmodified) after it is passed through either the `Conv2D` or the `Dense` layer.  

In [ ]:
type(x)

In [ ]:
# [batch_size, in_height, in_width, in_channels]
x.shape

##### Creating the Conv2D layer model

In [ ]:
model_conv = tf.keras.models.Sequential()
model_conv.add(
    conv_1x1(
        filters=NUM_OUTPUTS, 
        kernel_size=KERNEL_SIZE, 
        stride=STRIDE
    )
)

##### Creating the Dense layer model

In [ ]:
model_dense = tf.keras.models.Sequential()
model_dense.add(
    tf.keras.layers.Dense(
        units=NUM_OUTPUTS,
        kernel_initializer=custom_init
    )
)

##### Passing the input tensor through each model

In [ ]:
conv_out = model_conv(x)

In [ ]:
dense_out = model_dense(x)

##### Comparing the output shape

In [ ]:
conv_out.shape == dense_out.shape

### 2.2. Transposed Convolutions

In [ ]:
# TODO.

## 3. Closing Remarks

##### Alternatives
* TODO.

##### Extensions of task
* TODO.

## 4. Future Work

* TODO.

## Credits

This assignment was prepared by David Siller, Kelvin Lwin et al., 2020 (link [here]).

References
* TODO.


Helpful resources:
* TODO.